# TMBD Movie Database Project - Part 1
- Scotty Thomason

## MySQL database on Movies from a subset of IMDB's publicly available dataset

In [1]:
import pandas as pd
import numpy as np
import os,json

## Akas DataFrame

In [2]:
# akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
# akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
# akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [3]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [4]:
akas.replace({'\\N':np.nan}, inplace = True)

In [5]:
us_mask = (akas['region'] == 'US')
filter_akas = akas.loc[us_mask]

In [6]:
filter_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [7]:
filter_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408757 entries, 5 to 34697872
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1408757 non-null  object
 1   ordering         1408757 non-null  int64 
 2   title            1408757 non-null  object
 3   region           1408757 non-null  object
 4   language         3796 non-null     object
 5   types            970593 non-null   object
 6   attributes       45819 non-null    object
 7   isOriginalTitle  1407412 non-null  object
dtypes: int64(1), object(7)
memory usage: 96.7+ MB


In [8]:
## Save current dataframe to file.
# filter_akas.to_csv("Data/filtered_akas.csv.gz",compression='gzip',index=False)

## Basics DataFrame

In [9]:
#Data Sets
# basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [10]:
# Loading in the Data and saving it locally
# basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
# basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

In [11]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(filter_akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
9559978    False
9559979    False
9559980    False
9559981    False
9559982    False
Name: tconst, Length: 9559983, dtype: bool

In [12]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324305 entries, 0 to 9559938
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1324305 non-null  object
 1   titleType       1324305 non-null  object
 2   primaryTitle    1324305 non-null  object
 3   originalTitle   1324305 non-null  object
 4   isAdult         1324305 non-null  object
 5   startYear       1324305 non-null  object
 6   endYear         1324305 non-null  object
 7   runtimeMinutes  1324305 non-null  object
 8   genres          1324305 non-null  object
dtypes: object(9)
memory usage: 101.0+ MB


In [14]:
basics.replace({'\\N':np.nan}, inplace = True)

In [15]:
# Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace = True)
basics.dropna(subset=['genres'], inplace = True)

In [16]:
basics['titleType'].value_counts()

short           271405
movie           196125
tvEpisode       169471
video           113550
tvSeries         35619
tvMovie          25620
tvSpecial         8915
tvMiniSeries      5529
tvShort           4082
videoGame          197
Name: titleType, dtype: int64

In [17]:
mask1 = (basics['titleType'] == 'movie')
filter_basics = basics.loc[mask1]

In [18]:
mask2 = (filter_basics['startYear'] >= '2000') & (filter_basics['startYear'] < '2022')
filter_basics1 = filter_basics.loc[mask2]

In [19]:
filter_basics1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [20]:
# Exclude movies that are included in the documentary category.
is_documentary = filter_basics1['genres'].str.contains('documentary',case=False)
filter_basics2 = filter_basics1[~is_documentary]

In [21]:
filter_basics2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [22]:
filter_basics2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80484 entries, 34803 to 9559749
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          80484 non-null  object
 1   titleType       80484 non-null  object
 2   primaryTitle    80484 non-null  object
 3   originalTitle   80484 non-null  object
 4   isAdult         80484 non-null  object
 5   startYear       80484 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  80484 non-null  object
 8   genres          80484 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [23]:
## Save current dataframe to file.
# filter_basics2.to_csv("Data/filtered_basics.csv.gz",compression='gzip',index=False)

## Third DataFrame

In [24]:
# ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
# ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
# ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

In [25]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1948
1,tt0000002,5.8,264
2,tt0000003,6.5,1776
3,tt0000004,5.6,179
4,tt0000005,6.2,2581


In [26]:
ratings.replace({'\\N':np.nan}, inplace = True)

In [27]:
keepers = ratings['tconst'].isin(filter_akas['titleId'])
ratings = ratings[keepers]

In [28]:
## Save current dataframe to file.
# ratings.to_csv("Data/filtered_ratings.csv.gz",compression='gzip',index=False)

In [29]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486714 entries, 0 to 1274619
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         486714 non-null  object 
 1   averageRating  486714 non-null  float64
 2   numVotes       486714 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.9+ MB
